# Working asset trading bot that automatically places orders on an exchange called Oanda (we use a demo account and paper money) according to a random strategy. 

Note: the exchange is not 100% reliable so if the code doesn't run the first time, try again. Also, dependency issues may arise depending on the machine used.

API dependencies:
- oandapy
- v20

In [1]:
import sys
!{sys.executable} -m pip install git+https://github.com/oanda/oandapy.git
!{sys.executable} -m pip install git+https://github.com/oanda/v20-python.git
!{sys.executable} -m pip install git+https://github.com/oanda/v20-python-samples.git
!{sys.executable} -m pip install v20
import pandas as pd # Collect and manipulate historical data
import oandapy as opy # Implement basic API
import v20 # Implement rad API
import urllib # Acquire Oanda site xml data
import re # Discover available asset types and trading pairs
import random # Generate awesome numbers
import time # tick tock

  Cloning https://github.com/oanda/oandapy.git to c:\users\freddi~1\appdata\local\temp\pip-cffsgecv-build
  Cloning https://github.com/oanda/v20-python.git to c:\users\freddi~1\appdata\local\temp\pip-qwstd90h-build
    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\ProgramData\Anaconda3\lib\tokenize.py", line 452, in open
        buffer = _builtin_open(filename, 'rb')
    FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\FREDDI~1\\AppData\\Local\\Temp\\pip-qwstd90h-build\\setup.py'
    
    ----------------------------------------


Command "python setup.py egg_info" failed with error code 1 in C:\Users\FREDDI~1\AppData\Local\Temp\pip-qwstd90h-build\


  Cloning https://github.com/oanda/v20-python-samples.git to c:\users\freddi~1\appdata\local\temp\pip-w9k9eq18-build


In [2]:
class Oanda(object):
    """Pulls in custom data according to specs, converts to a Pandas DataFrame and cleans it!"""
    api = v20.Context('api-fxpractice.oanda.com','443',token='21928d31a473366fd2c27689849a3e6a-c81367d6ecf5be2be2a0b5c46c61bc3f')
    oanda = opy.API(environment='practice',access_token='21928d31a473366fd2c27689849a3e6a-c81367d6ecf5be2be2a0b5c46c61bc3f')
    def __init__(self, instrument, start, granularity, count):
        self.__accountID = '101-001-7843068-001' # unique to our paper trading account, does not change
        self.instrument = instrument # what asset pair to trade, i.e. EUR_USD
        self.start = start # start date for data collection
        self.granularity = granularity # how fine the data is (i.e. hourly, minutely)
        self.count = count # number of data points

    @property
    def attr(self):  
        return str(self.__accountID)

    @attr.setter
    def attr(self, value):
        self.__accountID = '101-001-7843068-001'

    def acquire_data(self): 
        # For example: Get EUR_USD data since May 15 2000, 5000 rows at 1 hr granularity (5000 hrs)
        data = Oanda.oanda.get_history(instrument = self.instrument, start = self.start, count = self.count, granularity = self.granularity)
        df = pd.DataFrame(data['candles']).set_index('time') # Converts data to Pandas DataFrame
        df.index = pd.DatetimeIndex(df.index) # Sets index to DateTime object

        end = str((df.index[-1]).date()) # next start date
        # Makes 5 API calls, each with <count> datapoints received, append DataFrames together
        for five_thousand_period in range(5): # useful if user wants more than max single call data
            data = Oanda.oanda.get_history(instrument = self.instrument, start = end, count = self.count, granularity = self.granularity)
            next_df = pd.DataFrame(data['candles']).set_index('time')
            next_df.index = pd.DatetimeIndex(next_df.index)
            end = str((next_df.index[-1]).date()) # next start date
            df = df.append(next_df)
        df.to_csv(self.instrument + '.csv') # turn into a csv
        
    def normalize_return_data(self):
        df = pd.DataFrame()
        df = df.from_csv(self.instrument + '.csv').drop_duplicates() # read from local dir and drop data collected for same timestamps
        # divide df by row 1 (norm) so first entry is 1 for all data and subsequent cells w/in columns are respective pct gains
        df['closeAsk'] = df['closeAsk'].divide(df['closeAsk'].iloc[0])
        print("Custom dataset loaded.")
        return df

instrument = input('What pair would you like to trade (i.e. EUR_USD): ')
start = input("Enter the start date of data you'd like to collect in the form of YYYY-MM-DD (i.e. 2000-05-15): ")
granularity = input("Enter the granularity of data you'd like to collect (i.e. H1): ")
count = int(input('Enter the number of data points you want to collect (i.e. 100): '))
ohana = Oanda(instrument, start, granularity, count)
ohana.acquire_data()
df = ohana.normalize_return_data()

What pair would you like to trade (i.e. EUR_USD): EUR_USD
Enter the start date of data you'd like to collect in the form of YYYY-MM-DD (i.e. 2000-05-15): 2000-05-15
Enter the granularity of data you'd like to collect (i.e. H1): H1
Enter the number of data points you want to collect (i.e. 100): 100
Custom dataset loaded.


In [3]:
class PlaceOrder(Oanda):
    """Finds tradeable pairs and places custom buy and sell orders!"""
    def __init__(self, instrument, start, granularity, count, Type):
        Oanda.__init__(self, instrument, start, granularity, count)
        self.Type = Type
    
    def get_pairs(self, Type, instrument): # check whether a given pair can be currently traded on Oanda
        xml = urllib.request.urlopen("https://www.oanda.com/forex-trading/markets/live").read().decode("utf-8") # opens URL with asset options
        forex = re.findall("[A-WY-Z]{3}/[A-WY-Z]{3}",xml) # finds all Forex trading pairs (none start with an 'X')
        metals = re.findall("[X][A-Z]{2}/[A-Z]{3}",xml) # finds all metals (they all start with an 'X')
        dct_of_trading_pairs = {"Forex": forex, "Metals": metals} # puts the above into a dict, values are lists
        return dct_of_trading_pairs[Type][29] # return EUR_USD for example
    
    def create_order(self, instrument, units): # Functionality: Place order of 100 units of pair with our account ID.
        response = Oanda.api.order.market(ohana.attr, instrument = instrument, units = units)
        print("Response: {} ({})".format(response.status, response.reason))
        
    def create_sell_order(self, instrument, units): # Functionality: Place sell order of 100 units of pair with our account ID.
        response = Oanda.api.order.market(ohana.attr, instrument = instrument, units = -units)
        print("Response: {} ({})".format(response.status, response.reason))

order = PlaceOrder("Forex", ohana.start, ohana.granularity, ohana.count, ohana.instrument)
order.get_pairs("Forex", ohana.instrument)
order.create_order(ohana.instrument, ohana.count)
order.create_sell_order(ohana.instrument, ohana.count)

Response: 201 (Created)
Response: 201 (Created)


In [6]:
class Strategy(Oanda):
    """Places 10 example orders according to random strategy to demonstrate program."""
    def __init__(self, instrument, start, granularity, count, df, time_wait):
        Oanda.__init__(self, instrument, start, granularity, count)
        self.df = df
        self.time_wait = time_wait

    def create_order(self, df, time_wait): # polymorphous function
        t0 = time.monotonic()
        # placeholder for profitable strategy:
        for i in range(self.time_wait):
            number = random.randrange(self.time_wait)
            t0 += 1
            time.sleep(t0 - time.monotonic())
            # to trade:
            if number > 4:
                order.create_order(instrument=self.instrument, units=self.count)
            elif number <= 4:
                order.create_sell_order(instrument=self.instrument, units=self.count)

strat = Strategy(ohana.instrument, ohana.start, ohana.granularity, ohana.count, ohana.normalize_return_data(), 10)
print("Generating 10 market buy and sell orders according to random strategy: ")
strat.create_order(ohana.normalize_return_data(), 10) # polymorphism, same name across classes

Custom dataset loaded.
Generating 10 market buy and sell orders according to random strategy: 
Custom dataset loaded.
Response: 201 (Created)
Response: 201 (Created)
Response: 201 (Created)
Response: 201 (Created)
Response: 201 (Created)
Response: 201 (Created)
Response: 201 (Created)
Response: 201 (Created)
Response: 201 (Created)
Response: 201 (Created)


In [5]:
class Log(Strategy):
    """Write order information to text file: Date, amount, price, ID, etc."""
    def __init__(self, account, prices):
        self.account = account
        self.prices = prices
    
    def get_acct(self):
        response = Oanda.api.account.get(ohana.attr)
        file = open("file.txt", "a")
        file.write(str(response.get(self.account, 200)))
        file.close()
        
    def get_prices(self): # Functionality: Check current price using GET
        response = Oanda.api.pricing.get(ohana.attr, instruments=ohana.instrument)
        file = open("file.txt", "a") 
        for price in response.get(self.prices, 200): # returns 'Price' object
            file.write(str(price))
        file.close()
        for price in response.get(self.prices, 200):
            return str(price)

    def read_txt(self):
        file = open("file.txt", "r") 
        return file.read() 
    
    def bid_ask_spread(self):
        bid_price_isolation = log.get_prices()[log.get_prices().find("bids:")+15:log.get_prices().find("bids:")+22]
        ask_price_isolation = log.get_prices()[log.get_prices().find("asks:")+15:log.get_prices().find("asks:")+22]
        file = open("file.txt", "a") 
        file.write("Bid ask spread is " + str(float(ask_price_isolation) - float(bid_price_isolation)))
        file.close()
        return float(ask_price_isolation) - float(bid_price_isolation)
    
    # operator overloading:
    def __sub__(self, other):
        return float(self) - float(other)
    
    def __str__(self): # CONVERT PRICE OR OTHER OANDA OBJECT TO STRING TO THEN WRITE TO FILE
        return str(self)
    
    def __repr__(self): # CONVERT PRICE TO CUSTOM STRING TO DISPLAY
        return "Price(" + str(self) + "')"
    
    def __float__(self):
        return float(self)
    
log = Log("account", "prices")
log.get_acct()
log.get_prices()
log.bid_ask_spread()

view = input('Type in L to view the log file, otherwise press enter: ').upper()
if view == 'L':
    print(log.read_txt())

Type in L to view the log file, otherwise press enter: L
id: 101-001-7843068-001
alias: Primary
currency: USD
balance: 99880.9294
createdByUserID: 7843068
createdTime: '2018-02-20T17:51:16.415341547Z'
pl: -94.845
resettablePL: -94.845
commission: 0.0
marginRate: 0.02
openTradeCount: 260
openPositionCount: 2
pendingOrderCount: 0
hedgingEnabled: false
unrealizedPL: -7678.01
NAV: 92202.9194
marginUsed: 15709.8058
marginAvailable: 76493.1136
positionValue: 533658.288
marginCloseoutUnrealizedPL: -7610.586
marginCloseoutNAV: 92270.3434
marginCloseoutMarginUsed: 15709.8058
marginCloseoutPercent: 0.08513
marginCloseoutPositionValue: 533658.288
withdrawalLimit: 76493.1136
marginCallMarginUsed: 15709.8058
marginCallPercent: 0.17026
lastTransactionID: '906'
trades:
- id: '163'
  instrument: EUR_USD
  price: 1.22342
  openTime: '2018-02-28T01:17:57.467420003Z'
  state: OPEN
  initialUnits: 100.0
  currentUnits: 100.0
  realizedPL: 0.0
  unrealizedPL: -0.422
  financing: -0.006
- id: '167'
  instru